In [362]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from numpy.random import default_rng
import pandas as pd
import random
questions = pd.read_csv("questions.csv", sep =";")

In [363]:
questions.head() #файл с вопросами

,1,2
0,Доход здесь и сейчас,Принести пользу обществу
1,Бизнес как семейное дело,Бизнес как развлечение
2,Доминирование на рынке,Получение прибыли
3,"Доход без рисков, простота в бизнесе","Задать тренд, изменить индустрию"
4,Преодоление препятствий на пути к бизнесу,Удовольствие от управления бизнесом


In [364]:
from numpy.random import default_rng
rng = default_rng()

Создание объектов для обучения:
[номера вопросов, ответы на них][взял ли кредит(0 или 1)] или [номера вопросов, нули(пропустил опросник), ответы, нули][0]



In [1524]:
X = np.array([np.concatenate((np.array(rng.choice(30, size= 7-i, replace= False))+1,
                                   np.zeros(i,dtype = int), 
                                   np.array(rng.choice(2, size = 6-i))+1,
                                      np.zeros(i,dtype = int),
                                   np.array([np.random.randint(2) + 1 if i == 0 else 0]),
                                    np.array([np.random.randint(2) if i ==0 else 0])))
                                   for i in rng.choice(7, 100000, p=[0.7,0.05 ,0.05, 0.05, 0.05, 0.05,0.05])]) #100к объектов, 30% бросили анкету


In [1525]:
X_, y_ = np.hsplit(X,[14])[0], np.hsplit(X,[14])[1].T
y_ = y_[0][X_[:,13] != 0]
X_ = X_[X_[:,13] != 0] #выбор прошедших анкету до конца для использования в обучении
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=40)

Модель размера 100000х14 .

In [1533]:
model = RandomForestClassifier(n_estimators = 4, max_depth = 8, random_state=1).fit(X_train,y_train)
prob = model.predict_proba(X_test)
#рассчет вероятности получить кредит на тестовой выборке

In [1534]:
probs = np.hstack((X_test, np.reshape(prob[:,1], (-1, 1))))

Учет штрафов от не доделанных анкет

In [1535]:
X_, y_ = np.hsplit(X,[14])[0], np.hsplit(X,[14])[1].T
drops = X_[X_[:,13] == 0]
for i in range(1,8):

    idx = np.where(drops[:,13-i] != 0)[0]
    for j in drops[idx]:
        probind1 = np.where(np.all(probs[:,7:13-i+1] == j[7:13-i+1],axis = 1))[0]
        probind2 = np.where(np.all(probs[probind1,:6 -i + 1] == j[:6-i+1],axis = 1))[0]
        for k in probind2:
            probs[k,14] = probs[k,14] * 0.95
    drops = np.delete(drops,idx,0)

Код работающий во время прохождения анкеты

In [1537]:
skip = False #индикатор того что вышли из опросника
obj = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0]) #рассматриваемый объект анкета
que = set(range(1,31)) #для избежания повтора вопросов
for i in range(1,8):
    #поиск похожих объектов в имеющейся базе
    if i == 1:
        samples = np.array([probs[probs[:,14].argsort()[::-1]][0]])
    else:
        samples = probs[np.all(probs[:,7:7+i-1] == obj[7:7+i-1],axis = 1)]
        samples = samples[np.all(samples[:,:i-1] == obj[:i-1],axis = 1)]
    
    if samples.size == 0:
        obj[i - 1] = np.random.choice(list(que)) #если таких еще не было то выдается рандомный вопрос
    else:
        obj[i - 1] = samples[samples[:,14].argsort()[::-1]][0,i-1] # выдается вопрос с предположительно наивышей вероятностью сподвигнуть взять кредит

    que.remove(obj[i - 1])
    
    obj[6+i] = np.random.choice(np.arange(0, 3), p=[0.05, 0.475, 0.475])#просто генерируется ответ или пропуск опросника
    if obj[6+i] == 0:
        skip = True
        
    if skip:
        if i == 1:
            ind = np.where(probs[:,0] == obj[0])[0]
        else:
            sampind = np.where(np.all(probs[:,7:7+i-1] == obj[7:7+i-1],axis = 1))[0]
            ind = np.where(np.all(probs[sampind,:i-1] == obj[:i-1],axis = 1))[0]
        for j in ind:
            probs[j,14] = probs[j,14]*0.95 #если анкету не прошли то штрафуем похожие последовательности вопросов
        break

if obj[-1] != 0:
    print(model.predict_proba([obj])[0][1]) #если опрос пройден до конца то можем посмотреть вероятность выдать креди        

0.5044415498091349
